# 구조화 V1

In [2]:
import pandas as pd
import re

pd.set_option('display.max_rows', 1000)

file_path = '2023_data.xlsx'
xls = pd.ExcelFile(file_path, engine='openpyxl')
sheets = {sheet_name: xls.parse(sheet_name, header=3) for sheet_name in xls.sheet_names}

def split_dataframe_by_sum(df, column_name):
    """
    주어진 DataFrame을 특정 열에서 '합계'를 기준으로 분할합니다.

    Parameters:
    df (pd.DataFrame): 분할할 DataFrame.
    column_name (str): '합계'를 검색할 열의 이름.

    Returns:
    list: '합계'를 포함한 각 부분으로 분할된 DataFrame의 리스트.
    """
    df.index = range(len(df))

    # 해당 열에서 '합계'가 있는 인덱스 찾기
    indices = df[df[column_name] == '합계'].index

    # DataFrame을 분할
    dataframes = []
    start_idx = 0

    for idx in indices:
        # '합계'를 포함하는 행까지 DataFrame 슬라이스
        dataframes.append(df.iloc[start_idx:idx+1])
        start_idx = idx + 1  # 다음 분할을 위해 시작 인덱스를 '합계' 다음 행으로 설정

    # 마지막 인덱스부터 DataFrame 끝까지 추가
    if start_idx < len(df):  # 마지막 '합계' 이후에도 데이터가 남아 있는 경우
        dataframes.append(df.iloc[start_idx:])

    return dataframes[0]

def split_dataframe_by_date(df, column_name):
    """
    주어진 DataFrame을 특정 열에서 '날짜'를 기준으로 분할합니다.

    Parameters:
    df (pd.DataFrame): 분할할 DataFrame.
    column_name (str): '날짜'를 검색할 열의 이름.

    Returns:
    list: '날짜'를 포함한 각 부분으로 분할된 DataFrame의 리스트.
    """
    df.index = range(len(df))
    df.columns = df.columns.str.replace('\n', '', regex=False)


    # 해당 열에서 '날짜'가 있는 인덱스 찾기
    indices = df[df[column_name] == '날짜'].index

    # DataFrame을 분할
    dataframes = []
    start_idx = 0

    for idx in indices+1:
        # '날짜'를 포함하는 행까지 DataFrame 슬라이스
        dataframes.append(df.iloc[start_idx:idx+1])
        start_idx = idx + 1  # 다음 분할을 위해 시작 인덱스를 '날짜' 다음 행으로 설정

    # 마지막 인덱스부터 DataFrame 끝까지 추가
    if start_idx < len(df):  # 마지막 '날짜' 이후에도 데이터가 남아 있는 경우
        dataframes.append(df.iloc[start_idx:])

    return dataframes[1]


# NaN 값을 이전 값으로 채우기
def fill_date(df):
    df['날짜'] = df['날짜'].fillna(method='ffill')

    return df

def decompose(df):
    # '1호기' 열에서 '2호기' 정확히 매치되는 패턴 검색
    df = df.rename(columns = {'1호기': '날짜'})
    pattern = r'^2호기$'
    mask = df['날짜'].apply(lambda x: bool(re.search(pattern, str(x))))
    indices = df[mask].index
    
    # DataFrame을 분할
    dataframes = []
    start_idx = 0
    
    for idx in indices:
        # 현재 인덱스를 새 DataFrame의 시작으로 포함
        dataframes.append(df.iloc[start_idx:idx])
        start_idx = idx
    
    # 마지막 인덱스부터 DataFrame 끝까지 추가
    dataframes.append(df.iloc[start_idx:])

    FirstUnit = dataframes[0]
    SecondUnit = dataframes[1]


    # 1호기 소계, 2호기 소계 추출
    monthly_first = SecondUnit[SecondUnit['날짜']=='1호기소계']
    montyly_second = SecondUnit[SecondUnit['날짜']=='2호기소계']

    # Dataframe 정규화
    FirstUnit = split_dataframe_by_sum(FirstUnit, '날짜')
    FirstUnit = split_dataframe_by_date(FirstUnit, '날짜')
    
    SecondUnit = split_dataframe_by_sum(SecondUnit, '날짜')
    SecondUnit = split_dataframe_by_date(SecondUnit, '날짜')

    # NaN 값을 이전 값으로 채워주기
    FirstUnit = fill_date(FirstUnit)
    SecondUnit = fill_date(SecondUnit)

    # 합계 Dataframe 추출
    FirstUnitTotalSum = FirstUnit[FirstUnit['생산규격']=='합계']
    SecondUnitTotalSum = SecondUnit[SecondUnit['생산규격']=='합계']
    
    return [FirstUnit, SecondUnit, monthly_first, montyly_second, FirstUnitTotalSum, SecondUnitTotalSum]

def monthly_df_maker(excel_sheets : dict) -> dict:
    '''
    montly_dict = {'1월' : montly_List, '2월' : montly_List, ...}
    montly_List = [1호기_df, 2호기_df, 1호기 소계, 2호기 소계, 1호기 합계, 2호기 합계]
    '''
    monthly_dict = {}

    try:
        for sheet in excel_sheets:
            print(sheet)
            result = decompose(excel_sheets[sheet])
            monthly_dict[sheet] = result
    except Exception as e :
        print(sheet)
        print(e)



    return monthly_dict


monthly_dict = monthly_df_maker(sheets)

1월
2월
3월
4월
5월
6월
7월
8월
9월
10월
11월
12월
평균
평균
list index out of range


# 구조화 V2

In [3]:
import pandas as pd
import re

pd.set_option('display.max_rows', 1000)

file_path = '2023_data.xlsx'
xls = pd.ExcelFile(file_path, engine='openpyxl')
sheets = {sheet_name: xls.parse(sheet_name, header=3) for sheet_name in xls.sheet_names}

def split_dataframe_by_sum(df, column_name):
    """
    주어진 DataFrame을 특정 열에서 '합계'를 기준으로 분할합니다.

    Parameters:
    df (pd.DataFrame): 분할할 DataFrame.
    column_name (str): '합계'를 검색할 열의 이름.

    Returns:
    list: '합계'를 포함한 각 부분으로 분할된 DataFrame의 리스트.
    """
    df.index = range(len(df))

    # 해당 열에서 '합계'가 있는 인덱스 찾기
    indices = df[df[column_name] == '합계'].index

    # DataFrame을 분할
    dataframes = []
    start_idx = 0

    for idx in indices:
        # '합계'를 포함하는 행까지 DataFrame 슬라이스
        dataframes.append(df.iloc[start_idx:idx+1])
        start_idx = idx + 1  # 다음 분할을 위해 시작 인덱스를 '합계' 다음 행으로 설정

    # 마지막 인덱스부터 DataFrame 끝까지 추가
    if start_idx < len(df):  # 마지막 '합계' 이후에도 데이터가 남아 있는 경우
        dataframes.append(df.iloc[start_idx:])

    return dataframes[0]

def split_dataframe_by_date(df, column_name):
    """
    주어진 DataFrame을 특정 열에서 '날짜'를 기준으로 분할합니다.

    Parameters:
    df (pd.DataFrame): 분할할 DataFrame.
    column_name (str): '날짜'를 검색할 열의 이름.

    Returns:
    list: '날짜'를 포함한 각 부분으로 분할된 DataFrame의 리스트.
    """
    df.index = range(len(df))
    df.columns = df.columns.str.replace('\n', '', regex=False)


    # 해당 열에서 '날짜'가 있는 인덱스 찾기
    indices = df[df[column_name] == '날짜'].index

    # DataFrame을 분할
    dataframes = []
    start_idx = 0

    for idx in indices+1:
        # '날짜'를 포함하는 행까지 DataFrame 슬라이스
        dataframes.append(df.iloc[start_idx:idx+1])
        start_idx = idx + 1  # 다음 분할을 위해 시작 인덱스를 '날짜' 다음 행으로 설정

    # 마지막 인덱스부터 DataFrame 끝까지 추가
    if start_idx < len(df):  # 마지막 '날짜' 이후에도 데이터가 남아 있는 경우
        dataframes.append(df.iloc[start_idx:])

    return dataframes[1]


# NaN 값을 이전 값으로 채우기
def fill_date(df):
    df['날짜'] = df['날짜'].fillna(method='ffill')

    return df


def convert_date_and_format(df, column, month):
    # 월을 나타내는 한글과 숫자를 매핑하는 딕셔너리
    month_dict = {'1월': '01', '2월': '02', '3월': '03', '4월': '04', '5월': '05', '6월': '06', 
                  '7월': '07', '8월': '08', '9월': '09', '10월': '10', '11월': '11', '12월': '12'}
    
    # 데이터 프레임의 복사본을 만듭니다.
    df_copy = df.copy()
    
    # 가정: df는 데이터 프레임, column은 변경하려는 열
    def convert_date(date, month):
        date = date.strip()
        if '야' in date:
            return '2023' + month_dict[month] + date.replace('야', '').replace('()', '').zfill(2) + ' 22:00'
        else:
            return '2023' + month_dict[month] + date.replace('()', '').zfill(2) + ' 12:00'

    df_copy[column] = df_copy[column].astype(str).apply(lambda x: convert_date(x, month))

    # 날짜 열의 데이터 타입을 datetime으로 변경
    df_copy[column] = pd.to_datetime(df_copy[column], format='%Y%m%d %H:%M', errors='coerce')
    return df_copy

def decompose(df, month):
    # '1호기' 열에서 '2호기' 정확히 매치되는 패턴 검색
    df = df.rename(columns = {'1호기': '날짜'})
    pattern = r'^2호기$'
    mask = df['날짜'].apply(lambda x: bool(re.search(pattern, str(x))))
    indices = df[mask].index
    
    # DataFrame을 분할
    dataframes = []
    start_idx = 0
    
    for idx in indices:
        # 현재 인덱스를 새 DataFrame의 시작으로 포함
        dataframes.append(df.iloc[start_idx:idx])
        start_idx = idx
    
    # 마지막 인덱스부터 DataFrame 끝까지 추가
    dataframes.append(df.iloc[start_idx:])

    FirstUnit = dataframes[0]
    SecondUnit = dataframes[1]


    # 1호기 소계, 2호기 소계 추출
    monthly_first = SecondUnit[SecondUnit['날짜']=='1호기소계']
    montyly_second = SecondUnit[SecondUnit['날짜']=='2호기소계']

    # Dataframe 정규화
    FirstUnit = split_dataframe_by_sum(FirstUnit, '날짜')
    FirstUnit = split_dataframe_by_date(FirstUnit, '날짜')
    
    SecondUnit = split_dataframe_by_sum(SecondUnit, '날짜')
    SecondUnit = split_dataframe_by_date(SecondUnit, '날짜')

    # NaN 값을 이전 값으로 채워주기
    FirstUnit = fill_date(FirstUnit)
    SecondUnit = fill_date(SecondUnit)

    # 합계 Dataframe 추출
    FirstUnitTotalSum = FirstUnit[FirstUnit['생산규격']=='합계']
    SecondUnitTotalSum = SecondUnit[SecondUnit['생산규격']=='합계']
    
    # 날짜 생성 및 포맷 변경
    FirstUnitTotalSum = convert_date_and_format(FirstUnitTotalSum, '날짜', month)
    SecondUnitTotalSum = convert_date_and_format(SecondUnitTotalSum, '날짜', month)
    
    return [FirstUnit, SecondUnit, monthly_first, montyly_second, FirstUnitTotalSum, SecondUnitTotalSum]

def monthly_df_maker(excel_sheets : dict) -> dict:
    '''
    montly_dict = {'1월' : montly_List, '2월' : montly_List, ...}
    montly_List = [1호기_df, 2호기_df, 1호기 소계, 2호기 소계, 1호기 합계, 2호기 합계]
    '''
    monthly_dict = {}

    try:
        for sheet in excel_sheets:
            print(sheet)
            result = decompose(excel_sheets[sheet],sheet)
            monthly_dict[sheet] = result
    except Exception as e :
        print(sheet)
        print(e)
        



    return monthly_dict


monthly_dict = monthly_df_maker(sheets)

1월
2월
3월
4월
5월
6월
7월
8월
9월
10월
11월
12월
평균
평균
list index out of range
